In [1]:
# Code to run in only in colab for packet download
!pip3 install gymnasium
!pip3 install gym[mujoco]


[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: /usr/local/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip
zsh:1: no matches found: gym[mujoco]


In [1]:
import gymnasium as gym
import math 
import os 
import torch
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
from collections import deque
import numpy as np
import tensorflow as tf

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.set_printoptions(threshold=10_000)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-27 19:21:28.597630: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DISCOUNT = 0.95
LEARNING_RATE = 1e-4

In [3]:
# test of the environment

env = gym.make('Walker2d-v4',render_mode="human") #change w "human" if needed
observation, info = env.reset()

for i in range(1000):
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()

env.close


OBSERVATIONS = env.observation_space.shape[0]
ACTIONS = 64



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/glfw/__init__.py:916: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


In [4]:
def discrete_to_continue_action(index):
    value = (2/ACTIONS) * index - 1
    return value

In [5]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer4 = nn.Linear(128, 128)
        self.layer5 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer4(x))
        x = F.relu(self.layer5(x))
        return self.layer3(x)
    
print(OBSERVATIONS)

main_nn = []
target_nn = []
optimizer_arr = []

for i in range(6):
    main_nn.append(DQN(OBSERVATIONS,ACTIONS).to(device))
    target_nn.append(DQN(OBSERVATIONS,ACTIONS).to(device))
    optimizer_arr.append(torch.optim.Adam(main_nn[i].parameters(), lr=LEARNING_RATE))

loss_fn = nn.MSELoss()

17


In [6]:
class ReplayMemory(object):

    def __init__(self, size, device ="cpu"):
        self.buffer = deque(maxlen=size)
        self.device = device
    
    def add(self,state,action, reward, next_state, done):
        self.buffer.append((state, action,reward,next_state,done))

    def __len__(self):
        return len(self.buffer)
    
    def sample(self, num_samples):
        states, actions, rewards, next_states, dones = [] , [] , [] , [] , []
        idx = np.random.choice(len(self.buffer), num_samples)
        for i in idx:
            elem = self.buffer[i]
            state, action, reward, next_state, done = elem
            states.append(np.array(state, copy=False))
            actions.append(np.array(action, copy=False))
            rewards.append(reward)
            next_states.append(np.array(next_state, copy=False))
            dones.append(done)
        states = np.array(states)
        actions = np.array(actions)
        rewards = np.array(rewards, dtype=np.float32)
        next_states = np.array(next_states)
        dones = np.array(dones, dtype=np.float32)
        return states, actions, rewards, next_states, dones
    
                    

In [10]:
def select_epsilon_greedy_action(state, epsilon):
    result = np.random.uniform()
    if result < epsilon:
        return env.action_space.sample()
    else:
        qs = []
        action_decimal = []
        for i in range(6):
            qs.append(main_nn[i](state).cpu().data.numpy())
            action_decimal.append((np.argmax(qs[i])*(2/ACTIONS))-1)
        return action_decimal

In [7]:
def training_step(states, actions, rewards, next_states, dones):
    max_next_qs = []
    target = []
    qs = []
    action_masks = []
    masked_qs = []
    loss = []
    for i in range(6):
        max_next_qs.append(target_nn[i](next_states).max(-1).values)
        target.append(rewards + (1.0 - dones) * DISCOUNT * max_next_qs[i])
        qs.append(main_nn[i](states))
        actions_transposed = torch.transpose(actions,0,1)
        action_masks.append(F.one_hot(actions_transposed[i].type(torch.int64), ACTIONS))
        masked_qs.append((action_masks[i] * qs[i]).sum(dim=-1))
        loss.append(loss_fn(masked_qs[i], target[i].detach()))
        optimizer_arr[i].zero_grad()
        loss[i].backward()
        optimizer_arr[i].step()
    return loss

In [112]:
# Hyperparameters.
num_episodes = 10000
epsilon = 0
batch_size = 32
discount = 0.99
buffer = ReplayMemory(100000, device=device)
cur_frame = 0
loss = 0

# Start training. Play game once and then train with a batch.
last_100_ep_rewards = []
for episode in range(num_episodes+1):
  state = env.reset()[0].astype(np.float32)
  ep_reward, done = 0, False
  while not done:
    state_in = torch.from_numpy(np.expand_dims(state, axis=0)).to(device)
    action = select_epsilon_greedy_action(state_in, epsilon)
    next_state, reward, done, info, unknown_attribute = env.step(action)
    next_state = next_state.astype(np.float32)
    ep_reward += reward
    # Save to experience replay.
    integer_action = []
    for i in range(6):
      integer_action.append((action[i] + 1)*(ACTIONS/2))
    buffer.add(state, integer_action, reward, next_state, done)
    state = next_state
    cur_frame += 1
    # Copy main_nn weights to target_nn.
    if cur_frame % 2000 == 0:
      for i in range(6):
        target_nn[i].load_state_dict(main_nn[i].state_dict())
    
    # Train neural network.
    if len(buffer) > batch_size:
      states, actions, rewards, next_states, dones = buffer.sample(batch_size)
      next_states = torch.from_numpy(next_states)
      rewards = torch.from_numpy(rewards)
      states = torch.from_numpy(states)
      dones = torch.from_numpy(dones)
      actions = torch.from_numpy(actions)


      loss = training_step(states, actions, rewards, next_states, dones)

  if episode < 950:
    epsilon -= 0.001

  if len(last_100_ep_rewards) == 100:
    last_100_ep_rewards = last_100_ep_rewards[1:]
  last_100_ep_rewards.append(ep_reward)

  if episode % 50 == 0:
    print(f'Episode {episode}/{num_episodes}. Epsilon: {epsilon:.3f}.'
          f' Reward in last 100 episodes: {np.mean(last_100_ep_rewards):.2f}, loss: {loss}')
    
env.close()

Episode 0/10000. Epsilon: -0.001. Reward in last 100 episodes: 1.99, loss: 0


KeyboardInterrupt: 

In [8]:
PATH = r"./2D/"
for i in range(6):
    main_nn[i].load_state_dict(torch.load(PATH + f"RNN{i}.param",map_location=device))

In [12]:
env_test = gym.make('Walker2d-v4',render_mode="human")

observation = env_test.reset()[0].astype(np.float32)


for i in range(100000):
    inp = torch.from_numpy(observation)
    action = select_epsilon_greedy_action(inp,0)
    observation, reward, terminated, truncated, info = env_test.step(action)
    observation = observation.astype(np.float32)

    

    if terminated or truncated:
        observation, info = env.reset()
        observation = observation.astype(np.float32)


env_test.close

: 

: 